# Clustering Toronto Neighborhood Data
## Data Scraping

Import libraries

In [1]:
# import libraries
from bs4 import BeautifulSoup
import requests

Use Beautiful Soup to extract the data from the Wikipedia page on Toronto neighborhoods

In [2]:
# set website
postcodes_html='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# pass html to BS
source = requests.get(postcodes_html).text
soup = BeautifulSoup(source, 'lxml')

# show in readable format
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":900271985,"wgRevisionId":900271985,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June",

Pull out the table with the neighborhood data

In [3]:
full_table = soup.find('table', class_='wikitable sortable')
print(full_table)

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

Loop through each row in the table then through each cell to get each parameter

In [4]:
for table_row in full_table.find_all('tr'):
	counter = 1
	for table_cell in table_row.find_all('td'):
		if counter == 1:
			postcode = table_cell.text
			print("Postcode is "+postcode )
		elif counter == 2:
			borough = table_cell.text
			print("Borough is "+borough )
		else:
			neighborhood = table_cell.text
			print("Neighborhood is "+neighborhood )
		counter = counter + 1

Postcode is M1A
Borough is Not assigned
Neighborhood is Not assigned

Postcode is M2A
Borough is Not assigned
Neighborhood is Not assigned

Postcode is M3A
Borough is North York
Neighborhood is Parkwoods

Postcode is M4A
Borough is North York
Neighborhood is Victoria Village

Postcode is M5A
Borough is Downtown Toronto
Neighborhood is Harbourfront

Postcode is M5A
Borough is Downtown Toronto
Neighborhood is Regent Park

Postcode is M6A
Borough is North York
Neighborhood is Lawrence Heights

Postcode is M6A
Borough is North York
Neighborhood is Lawrence Manor

Postcode is M7A
Borough is Queen's Park
Neighborhood is Not assigned

Postcode is M8A
Borough is Not assigned
Neighborhood is Not assigned

Postcode is M9A
Borough is Etobicoke
Neighborhood is Islington Avenue

Postcode is M1B
Borough is Scarborough
Neighborhood is Rouge

Postcode is M1B
Borough is Scarborough
Neighborhood is Malvern

Postcode is M2B
Borough is Not assigned
Neighborhood is Not assigned

Postcode is M3B
Borough is 

Create an empty pandas dataframe to store the neighborhood data

In [5]:
# create empty dataframe
import pandas as pd
column_names = ['Postcode', 'Borough', 'Neighborhood'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
neighborhoods.head()

,Postcode,Borough,Neighborhood


Use the above loop to load the neighborhood data with these caveats:
1. Ignore cells with a borough that is Not assigned.
2. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
# load data into dataframe
for table_row in full_table.find_all('tr'):
	counter = 1
	postcode = 'NA'
	borough = 'NA'
	neighborhood = 'NA'
	for table_cell in table_row.find_all('td'):
		if counter == 1:
			postcode = table_cell.text
			print("Postcode is "+postcode )
		elif counter == 2:
			borough = table_cell.text
			print("Borough is "+borough )
		else:
			neighborhood = table_cell.text.strip('\n')
			print("Neighborhood is "+neighborhood )
		counter = counter + 1
	print('The postcode is {}, borough is {}, and neighborhood is {}.'.format(postcode, borough, neighborhood))
	if (postcode == 'NA') or (borough == 'Not assigned'):
		print('Skipping this row')
	elif neighborhood == 'Not assigned':
		neighborhoods = neighborhoods.append({'Postcode':postcode,
		                                      'Borough': borough,
											  'Neighborhood': borough}, ignore_index=True)
	else:
		neighborhoods = neighborhoods.append({'Postcode':postcode,
		                                      'Borough': borough,
											  'Neighborhood': neighborhood}, ignore_index=True)

The postcode is NA, borough is NA, and neighborhood is NA.
Skipping this row
Postcode is M1A
Borough is Not assigned
Neighborhood is Not assigned
The postcode is M1A, borough is Not assigned, and neighborhood is Not assigned.
Skipping this row
Postcode is M2A
Borough is Not assigned
Neighborhood is Not assigned
The postcode is M2A, borough is Not assigned, and neighborhood is Not assigned.
Skipping this row
Postcode is M3A
Borough is North York
Neighborhood is Parkwoods
The postcode is M3A, borough is North York, and neighborhood is Parkwoods.
Postcode is M4A
Borough is North York
Neighborhood is Victoria Village
The postcode is M4A, borough is North York, and neighborhood is Victoria Village.
Postcode is M5A
Borough is Downtown Toronto
Neighborhood is Harbourfront
The postcode is M5A, borough is Downtown Toronto, and neighborhood is Harbourfront.
Postcode is M5A
Borough is Downtown Toronto
Neighborhood is Regent Park
The postcode is M5A, borough is Downtown Toronto, and neighborhood i

Postcode is M7J
Borough is Not assigned
Neighborhood is Not assigned
The postcode is M7J, borough is Not assigned, and neighborhood is Not assigned.
Skipping this row
Postcode is M8J
Borough is Not assigned
Neighborhood is Not assigned
The postcode is M8J, borough is Not assigned, and neighborhood is Not assigned.
Skipping this row
Postcode is M9J
Borough is Not assigned
Neighborhood is Not assigned
The postcode is M9J, borough is Not assigned, and neighborhood is Not assigned.
Skipping this row
Postcode is M1K
Borough is Scarborough
Neighborhood is East Birchmount Park
The postcode is M1K, borough is Scarborough, and neighborhood is East Birchmount Park.
Postcode is M1K
Borough is Scarborough
Neighborhood is Ionview
The postcode is M1K, borough is Scarborough, and neighborhood is Ionview.
Postcode is M1K
Borough is Scarborough
Neighborhood is Kennedy Park
The postcode is M1K, borough is Scarborough, and neighborhood is Kennedy Park.
Postcode is M2K
Borough is North York
Neighborhood i

Postcode is M9R
Borough is Etobicoke
Neighborhood is St. Phillips
The postcode is M9R, borough is Etobicoke, and neighborhood is St. Phillips.
Postcode is M1S
Borough is Scarborough
Neighborhood is Agincourt
The postcode is M1S, borough is Scarborough, and neighborhood is Agincourt.
Postcode is M2S
Borough is Not assigned
Neighborhood is Not assigned
The postcode is M2S, borough is Not assigned, and neighborhood is Not assigned.
Skipping this row
Postcode is M3S
Borough is Not assigned
Neighborhood is Not assigned
The postcode is M3S, borough is Not assigned, and neighborhood is Not assigned.
Skipping this row
Postcode is M4S
Borough is Central Toronto
Neighborhood is Davisville
The postcode is M4S, borough is Central Toronto, and neighborhood is Davisville.
Postcode is M5S
Borough is Downtown Toronto
Neighborhood is Harbord
The postcode is M5S, borough is Downtown Toronto, and neighborhood is Harbord.
Postcode is M5S
Borough is Downtown Toronto
Neighborhood is University of Toronto
Th

Neighborhood is Not assigned
The postcode is M7X, borough is Not assigned, and neighborhood is Not assigned.
Skipping this row
Postcode is M8X
Borough is Etobicoke
Neighborhood is The Kingsway
The postcode is M8X, borough is Etobicoke, and neighborhood is The Kingsway.
Postcode is M8X
Borough is Etobicoke
Neighborhood is Montgomery Road
The postcode is M8X, borough is Etobicoke, and neighborhood is Montgomery Road.
Postcode is M8X
Borough is Etobicoke
Neighborhood is Old Mill North
The postcode is M8X, borough is Etobicoke, and neighborhood is Old Mill North.
Postcode is M9X
Borough is Not assigned
Neighborhood is Not assigned
The postcode is M9X, borough is Not assigned, and neighborhood is Not assigned.
Skipping this row
Postcode is M1Y
Borough is Not assigned
Neighborhood is Not assigned
The postcode is M1Y, borough is Not assigned, and neighborhood is Not assigned.
Skipping this row
Postcode is M2Y
Borough is Not assigned
Neighborhood is Not assigned
The postcode is M2Y, borough is

Show the top five rows of the dataframe.

In [8]:
neighborhoods.head(5)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Show the size of the dataframe.

In [9]:
neighborhoods.shape

(211, 3)

## Geo Locations

Install and import geocoder

In [10]:
# import geocoder
! pip install geocoder
import geocoder

Create a new dataframe with geo location columns

In [11]:
# create a load new dataframe
column_names = ['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
hoods_ll = pd.DataFrame(columns=column_names)
hoods_ll.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude


Load data into the dataframe

In [12]:
# load data into dataframe
for table_row in full_table.find_all('tr'):
	counter = 1
	postcode = 'NA'
	borough = 'NA'
	neighborhood = 'NA'
	for table_cell in table_row.find_all('td'):
		if counter == 1:
			postcode = table_cell.text
		elif counter == 2:
			borough = table_cell.text
		else:
			neighborhood = table_cell.text.strip('\n')
		counter = counter + 1
	if (postcode == 'NA') or (borough == 'Not assigned'):
		print('Skipping this row')
	elif neighborhood == 'Not assigned':
		hoods_ll = hoods_ll.append({'Postcode':postcode,
		                                      'Borough': borough,
											  'Neighborhood': borough}, ignore_index=True)
	else:
		hoods_ll = hoods_ll.append({'Postcode':postcode,
		                                      'Borough': borough,
											  'Neighborhood': neighborhood}, ignore_index=True)
											  
hoods_ll.head(5)

Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping this row
Skipping t

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,Harbourfront,NaN,NaN
3,M5A,Downtown Toronto,Regent Park,NaN,NaN
4,M6A,North York,Lawrence Heights,NaN,NaN


Using geocoder just hangs, even when I try to look up a single postal code. Instead, I am importing data from the CSV file provided and saving it into a dataframe.

In [13]:
# Read data from CSV file into a dataframe
csv_path = 'http://cocl.us/Geospatial_data'
geo_df = pd.read_csv(csv_path)	
geo_df.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Match each row in the neighborhood dataframe with the postal code that matches in the dataframe of geolocation data. Then save the latitude and longitude into the neighborhood row.

In [14]:
for i in hoods_ll.index:
	# set postal code
	postal_code = hoods_ll.loc[i, 'Postcode']
	
	latitude = geo_df.loc[geo_df['Postal Code'] == postal_code].Latitude.values[0]
	longitude = geo_df.loc[geo_df['Postal Code'] == postal_code].Longitude.values[0]
	# print('The location of {} is {}, {}'.format(postal_code, latitude, longitude))

	hoods_ll.at[i, 'Latitude'] = latitude
	hoods_ll.at[i, 'Longitude'] = longitude

hoods_ll.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
3,M5A,Downtown Toronto,Regent Park,43.6543,-79.3606
4,M6A,North York,Lawrence Heights,43.7185,-79.4648
5,M6A,North York,Lawrence Manor,43.7185,-79.4648
6,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
7,M9A,Etobicoke,Islington Avenue,43.6679,-79.5322
8,M1B,Scarborough,Rouge,43.8067,-79.1944
9,M1B,Scarborough,Malvern,43.8067,-79.1944


## Data Analysis

Import required libraries

In [15]:
# import libraries
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Create a new dataframe for just boroughs with Toronto in the name

In [16]:
# slice df to create new df for just Toronto
toronto_data = hoods_ll[hoods_ll['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
1,M5A,Downtown Toronto,Regent Park,43.6543,-79.3606
2,M5B,Downtown Toronto,Ryerson,43.6572,-79.3789
3,M5B,Downtown Toronto,Garden District,43.6572,-79.3789
4,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754


Get geolocation data for Toronto

In [17]:
# get coordinates for Manhattan
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto with markers for the neighborhoods

In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define variables for using the Foursquare API

In [19]:
# define Foursquare variables
CLIENT_ID = 'CT3K4Z2AEBTWGOKQLQKZ135JJ3B44KOQTB4BMEJ4R0AXXWSD' # your Foursquare ID
CLIENT_SECRET = 'TSJKVMWFPKG2ZV4NYZQDVIR5FIYOSRICDTHHKHWSMV5JHDMZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Create a function to pull venue information for a given neighborhood

In [20]:
# create a function to repeat the same process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Call the above function and store the venue information into a new dataframe

In [21]:
# set limits for Foursquare calls
radius = 500
LIMIT=100

# write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

# check size of resulting df
print(toronto_venues.shape)

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


Show how many venues were found for each neighborhood

In [22]:
# how many venues for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,14,14,14,14,14,14
Berczy Park,55,55,55,55,55,55
Brockton,21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
CN Tower,14,14,14,14,14,14
Cabbagetown,46,46,46,46,46,46
Central Bay Street,88,88,88,88,88,88
Chinatown,100,100,100,100,100,100


How many unique categories did we find?

In [23]:
# how many unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


## Analyze each neighborhood

Create new dataframe for performing one-hot encoding

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Add Neighborhood data as the first column

In [25]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

Group by the mean of each category

In [26]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.010000,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.030000,0.010000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.060000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0100

Show the top 5 categories for each neighborhood

In [27]:
# show top 5 categories for each neighborhood
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3                  Bar  0.04
4  American Restaurant  0.04


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3          Boutique  0.07
4  Sculpture Garden  0.07


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.04
3  Italian Restaurant  0.04
4      Farmers Market  0.04


----Brockton----
            venue  freq
0  Breakfast Spot  0.10
1            Café  0.10
2     Coffee Shop  0.10
3     Yoga Studio  0.05
4             Gym  0.05


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0       Yoga Studio  0.05
1     Auto Workshop  0.05
2              Park  0.05
3        Comic Shop  0.05
4  Recording Studio  0.05


----CN Tower----
              venue  freq
0    Airport Lo

                venue  freq
0                 Pub  0.14
1         Coffee Shop  0.14
2  Light Rail Station  0.07
3         Pizza Place  0.07
4          Bagel Shop  0.07


----Regent Park----
                venue  freq
0         Coffee Shop  0.17
1                Park  0.06
2              Bakery  0.06
3                 Pub  0.06
4  Mexican Restaurant  0.04


----Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3                  Bar  0.04
4  American Restaurant  0.04


----Riverdale----
                    venue  freq
0        Greek Restaurant  0.21
1             Coffee Shop  0.10
2      Italian Restaurant  0.07
3          Ice Cream Shop  0.07
4  Furniture / Home Store  0.05


----Roncesvalles----
            venue  freq
0       Gift Shop  0.13
1  Breakfast Spot  0.13
2       Bookstore  0.07
3    Dessert Shop  0.07
4         Dog Run  0.07


----Rosedale----
         venue  freq
0         Park  0.50
1   Playg

Create function to sort venues in descending order

In [28]:
# sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create columns for a new dataframe with top 10 venue categories

In [29]:
# create columns according to number of top venue categories
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

Create a new dataframe with the columns we defined and load data into it

In [30]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

In [31]:
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Bar,Burger Joint,Hotel,Bakery,Cosmetics Shop
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Airport,Airport Food Court,Airport Gate,Plane
2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Cheese Shop,Farmers Market,Beer Bar,Steakhouse,Bakery,Italian Restaurant
3,Brockton,Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Pet Store,Climbing Gym,Restaurant,Caribbean Restaurant,Burrito Place
4,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Pizza Place,Gym / Fitness Center,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery,Skate Park


## k-means Clustering

Set the number of clusters

In [32]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

Run k-means clustering on our data

In [33]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

Check the cluster labels for the first 10 rows

In [34]:
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 2, 0, 0, 0, 0])

Add cluster lables to our neighborhood dataframe

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data

Merge all our data together

In [36]:
# merge data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606,0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Café,Gym / Fitness Center
1,M5A,Downtown Toronto,Regent Park,43.6543,-79.3606,0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Café,Gym / Fitness Center
2,M5B,Downtown Toronto,Ryerson,43.6572,-79.3789,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Tea Room
3,M5B,Downtown Toronto,Garden District,43.6572,-79.3789,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Tea Room
4,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754,0,Coffee Shop,Café,Restaurant,Bakery,Hotel,Cosmetics Shop,Breakfast Spot,Pizza Place,Gastropub,Park


Set parameters for mapping the results

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

Add neighborhood cluster markers to the map and visualize it

In [38]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters